# 构建同学兴趣爱好图数据库


先对数据进行建模: 

- 实体: 同学, 爱好
- 关系: "喜欢做"

因此, 图数据库中应包含 "同学" 结点, 以及 "爱好" 结点, 其间通过 "喜欢做" 进行关联.

```cypher
CREATE (p:Person { name: 'Xiaoming' })-[:LIKES]->(h:Hobby { name: '唱歌' })
```

## 数据载入

同学兴趣爱好表格原始数据类似如下:

| name | id | hobbies |
| :--- | :--- | :--- |
| ... | ... | ... |
| 小明 | 0000001 | 唱歌 听音乐 看电视剧 看小说 追星 |
| 小红 | 0000002 | 音乐，睡觉 |
| ... | ... | ... |

将表格另存为 CSV 格式后, 可以载入到 Python 中. 这里使用 `csv.DictReader` 将信息读取为字典的形式:


In [ ]:
import csv

with open('hobbies.csv') as f:
    data = [record for record in csv.DictReader(f)]

In [ ]:
data

## 内容处理

留意到 `hobbies` 字段的文本由多个字段组成，但是每项记录中各字段间采用的分隔符各不相同，可以采用 `re.split()` 方法进行处理。


分割字段示例代码: 

In [ ]:
import re

for record in data:
    # 分割字段
    record['hobbies'] = re.split('，|、|\ |；|, |;', record['hobbies'])
    # 去除空项
    record['hobbies'] = [token for token in record['hobbies'] if len(token) > 0]


但留意到存在类似“游戏小说”这样将两项汉语词汇合并写成一个词的, 需要分割开来, 可以使用 `jieba.lcut()` 方法, 参考:

In [ ]:
import jieba

jieba.lcut('游戏小说')

显示处理后的 `data`:

In [ ]:
data

## 提取兴趣爱好关键词

总结出爱好中所有出现过的项目, 存储在集合 `hobbies` 中.

In [ ]:
hobbies = set()

for record in data:
    for token in record['hobbies']:
        for word in jieba.lcut(token):
            hobbies.add(word)

hobbies

集合中包含一些动词和一些无意义的词, 这里可以手动去除.

先查看有哪些词:


In [ ]:

[word for word in hobbies if len(word) == 1]

去除一些词:

In [ ]:
hobbies.remove('和')
hobbies.remove('与')
hobbies.remove('在')
hobbies.remove('看')
hobbies.remove('想')
hobbies.remove('打')
hobbies.remove('上')
hobbies.remove('听')
hobbies.remove('站')
hobbies.remove('里')
hobbies.remove('刷')
hobbies.remove('b')
hobbies.remove('一个')

In [ ]:
hobbies

## 将内容写出为 CSV

In [ ]:
rows = []

for person in data:
    for token in person['hobbies']:
        for item in hobbies:
            if token.find(item) != -1:
                rows.append({
                    'name': person['name'],
                    'id': person['id'],
                    'likes': item,
                })

rows

In [ ]:
with open('hobbies-for-import.csv', 'w') as f:
    dict_writer = csv.DictWriter(f, fieldnames=rows[0].keys(), dialect=csv.excel)
    dict_writer.writeheader()
    dict_writer.writerows(rows)

## 导入到 Neo4j 数据库

Import to Neo4j

```cypher
LOAD CSV WITH HEADERS FROM 'file:///hobbies-for-import.csv' AS row 
MERGE (p:Person {name: row.name, id: row.id}) 
WITH p, row 
MERGE (hobby:Hobby {name: row.likes}) 
MERGE (p)-[r:LIKES]->(hobby)
```